In [1]:
!pip install termcolor
!pip install openai==0.28


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sqlite3
import json
import pandas as pd
import os
import logging
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import openai
from termcolor import colored

In [ ]:
import sqlite3
import json
import pandas as pd
import os
import logging
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import openai
from termcolor import colored


class RealEstateAnalyzer:
    def __init__(self, db_path: str, log_path: str = "logs"):
        """Inicializa el analizador de propiedades inmobiliarias."""
        self._setup_logging(log_path)
        self._validate_and_connect_db(db_path)
        self.schema = self._get_schema()
        self._init_query_mappings()

    def _validate_and_connect_db(self, db_path: str):
        """Valida y establece la conexión a la base de datos."""
        try:
            if not os.path.exists(db_path):
                self.logger.error(f"Base de datos no encontrada en: {db_path}")
                raise FileNotFoundError(f"Base de datos no encontrada en: {db_path}")
            
            self.conn = sqlite3.connect(db_path)
            self.logger.info(f"Conexión exitosa a la base de datos: {db_path}")
            
            cursor = self.conn.cursor()
            cursor.execute("""
                SELECT name FROM sqlite_master 
                WHERE type='table' AND name='chat_property';
            """)
            
            if not cursor.fetchone():
                self.logger.error("La tabla chat_property no existe en la base de datos")
                raise Exception("La tabla chat_property no existe en la base de datos")
            
            cursor.execute("PRAGMA table_info(chat_property)")
            columns = {col[1] for col in cursor.fetchall()}
            required_columns = {
                'id', 'location', 'price', 'square_meters', 'property_type',
                'num_bedrooms', 'num_rooms', 'project_type'
            }
            
            missing_columns = required_columns - columns
            if missing_columns:
                self.logger.error(f"Columnas faltantes en chat_property: {missing_columns}")
                raise Exception(f"Columnas faltantes en chat_property: {missing_columns}")
                
        except sqlite3.Error as e:
            self.logger.error(f"Error SQLite al conectar con la base de datos: {str(e)}")
            raise
        except Exception as e:
            self.logger.error(f"Error al validar la base de datos: {str(e)}")
            raise

    def _setup_logging(self, log_path: str):
        """Configura el sistema de logging."""
        if not os.path.exists(log_path):
            os.makedirs(log_path)
        
        log_file = os.path.join(log_path, f'real_estate_{datetime.now().strftime("%Y%m%d")}.log')
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s [%(levelname)s] %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def _get_schema(self) -> str:
        """Obtiene el esquema de la base de datos."""
        cursor = self.conn.cursor()
        schema_str = "Esquema de la Base de Datos:\n"
        
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' 
            AND name NOT LIKE 'sqlite_%'
            AND name NOT LIKE 'django_%'
            AND name NOT LIKE 'auth_%';
        """)
        
        for (table_name,) in cursor.fetchall():
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            
            schema_str += f"\nTabla: {table_name}\nColumnas:\n"
            for col in columns:
                schema_str += f"  - {col[1]} ({col[2]})\n"
                
        return schema_str

    def _init_query_mappings(self):
        """Inicializa las categorías y consultas predefinidas."""
        self.categorias = {
            "identificacion_localizacion": [
                "identificador", "id", "ubicacion", "localización", "ciudad",
                "provincia", "pais", "zona", "barrio", "sector"
            ],
            "analisis_precio": [
                "precio", "valor", "costo", "mercado", "comparable",
                "metro cuadrado", "competitivo", "valoración", "tasacion",
                "avaluo", "comprable", "similar"
            ],
            "detalles_propiedad": [
                "habitaciones", "dormitorios", "cuartos", "espacios",
                "distribución", "características", "condición", "baños",
                "features", "amenities", "comodidades"
            ],
            "clasificacion_categoria": [
                "categoría", "tipo", "proyecto", "residencial", "comercial",
                "lujo", "económico", "construcción", "remodelación",
                "clase", "premium", "estándar"
            ],
            "cronologia": [
                "tiempo", "mercado", "fecha", "histórico", "antigüedad",
                "created", "cuando", "desde", "hasta", "periodo"
            ]
        }

        self.consultas = {
            "identificacion_localizacion": {
                "info_basica": """
                    SELECT 
                        id,
                        location,
                        property_type,
                        project_category,
                        created_at,
                        ROUND(price, 2) as price,
                        square_meters
                    FROM chat_property
                    WHERE {condition}
                """,
                "analisis_ubicacion": """
                    SELECT 
                        location,
                        COUNT(*) as total_propiedades,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(DISTINCT property_type) as tipos_propiedad,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio
                    FROM chat_property
                    GROUP BY location
                    ORDER BY total_propiedades DESC
                """
            },
            "analisis_precio": {
                "comparativa_precios": """
                    SELECT 
                        location,
                        property_type,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(MIN(price), 2) as precio_minimo,
                        ROUND(MAX(price), 2) as precio_maximo,
                        ROUND(AVG(price/square_meters), 2) as precio_m2,
                        COUNT(*) as total_propiedades
                    FROM chat_property
                    GROUP BY location, property_type
                    ORDER BY precio_promedio DESC
                """,
                "analisis_competitividad": """
                    SELECT 
                        p1.property_type,
                        p1.location,
                        ROUND(p1.price, 2) as price,
                        p1.square_meters,
                        ROUND(p1.price/p1.square_meters, 2) as precio_m2,
                        ROUND(AVG(p2.price/p2.square_meters), 2) as precio_m2_promedio_zona,
                        ROUND((p1.price/p1.square_meters - AVG(p2.price/p2.square_meters)) / 
                            AVG(p2.price/p2.square_meters) * 100, 2) as diferencia_porcentual
                    FROM chat_property p1
                    LEFT JOIN chat_property p2 
                    ON p2.location = p1.location 
                    AND p2.property_type = p1.property_type
                    GROUP BY p1.id
                    ORDER BY p1.location, p1.property_type
                """,
                "analisis_tendencia": """
                    SELECT 
                        strftime('%Y-%m', created_at) as periodo,
                        location,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        COUNT(*) as total_propiedades
                    FROM chat_property
                    GROUP BY periodo, location
                    ORDER BY periodo DESC, precio_promedio DESC
                """
            },
            "detalles_propiedad": {
                "analisis_espacios": """
                    SELECT 
                        property_type,
                        num_bedrooms,
                        num_rooms,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio
                    FROM chat_property
                    GROUP BY property_type, num_bedrooms, num_rooms
                    ORDER BY property_type, num_bedrooms
                """,
                "detalles_completos": """
                    SELECT 
                        id,
                        property_type,
                        location,
                        num_bedrooms,
                        num_rooms,
                        square_meters,
                        description,
                        project_category,
                        project_type,
                        residence_type,
                        ROUND(price, 2) as price,
                        ROUND(price/square_meters, 2) as precio_m2,
                        created_at
                    FROM chat_property
                    WHERE {condition}
                """
            },
            "clasificacion_categoria": {
                "distribucion_categorias": """
                    SELECT 
                        project_category,
                        project_type,
                        residence_type,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio
                    FROM chat_property
                    GROUP BY project_category, project_type, residence_type
                    HAVING cantidad > 0
                    ORDER BY cantidad DESC
                """,
                "analisis_tipos": """
                    SELECT 
                        property_type,
                        project_type,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        ROUND(MIN(price), 2) as precio_minimo,
                        ROUND(MAX(price), 2) as precio_maximo,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio
                    FROM chat_property
                    GROUP BY property_type, project_type
                    HAVING cantidad > 0
                    ORDER BY cantidad DESC
                """
            },
            "cronologia": {
                "analisis_temporal": """
                    SELECT 
                        strftime('%Y-%m', created_at) as mes,
                        COUNT(*) as nuevas_propiedades,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(DISTINCT property_type) as tipos_diferentes,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio
                    FROM chat_property
                    GROUP BY mes
                    ORDER BY mes DESC
                """
            }
        }

    def _extraer_ubicaciones(self, pregunta: str) -> List[str]:
        """Extrae las ubicaciones mencionadas en la pregunta."""
        ubicaciones_conocidas = [
            "Mar de Plata", "Buenos Aires", "Villavicencio", "Medellin",
            "Recoleta", "Palermo", "Belgrano", "Mar del Plata"
        ]
        
        found = []
        pregunta_lower = pregunta.lower()
        for ubicacion in ubicaciones_conocidas:
            if ubicacion.lower() in pregunta_lower:
                found.append(ubicacion)
        
        return found

    def _extraer_tipos_propiedad(self, pregunta: str) -> List[str]:
        """Extrae los tipos de propiedad mencionados en la pregunta."""
        tipos_conocidos = {
            "apartamento": "apartment",
            "casa": "house",
            "cabaña": "cabin",
            "departamento": "apartment",
            "oficina": "office",
            "local": "commercial",
            "propiedad": "property"
        }
        
        found = []
        pregunta_lower = pregunta.lower()
        for tipo_es, tipo_en in tipos_conocidos.items():
            if tipo_es in pregunta_lower:
                found.append(tipo_en)
        
        return found

    def _format_raw_response(self, datos: Dict[str, Any]) -> str:
        """Formatea los datos en caso de error con GPT."""
        response = "📊 Datos encontrados:\n\n"
        
        for nombre_consulta, resultados in datos.items():
            if isinstance(resultados, list) and resultados:
                response += f"📍 {nombre_consulta.replace('_', ' ').title()}:\n"
                for item in resultados[:5]:
                    response += f"  • {str(item)}\n"
                response += "\n"
            elif isinstance(resultados, dict) and "error" in resultados:
                response += f"❌ Error en {nombre_consulta}: {resultados['error']}\n\n"
        
        return response.strip()

    def _construir_prompt(self, pregunta: str, datos: Dict[str, Any], categoria: str) -> str:
        """Construye el prompt para GPT basado en los datos."""
        prompt = f"""
        Pregunta del usuario: {pregunta}
        
        Categoría identificada: {categoria}
        
        Datos disponibles:
        """
        
        for nombre_consulta, resultados in datos.items():
            prompt += f"\n{nombre_consulta.upper()}:\n"
            if isinstance(resultados, list):
                for item in resultados[:5]:
                    prompt += json.dumps(item, ensure_ascii=False, indent=2) + "\n"
            elif isinstance(resultados, dict) and "error" in resultados:
                prompt += f"Error: {resultados['error']}\n"
        
        prompt += """
        Por favor, proporciona un análisis detallado que:
        1. Responda directamente a la pregunta del usuario
        2. Destaque insights relevantes de los datos
        3. Proporcione contexto cuando sea necesario
        4. Ofrezca recomendaciones basadas en los datos
        5. Incluya métricas específicas cuando estén disponibles
        6. Compare con promedios o referencias del mercado cuando sea posible
        """
        
        return prompt

    def _get_system_prompt(self, categoria: str) -> str:
        """Obtiene el prompt de sistema según la categoría."""
        prompts = {
            "identificacion_localizacion": """Eres un experto en análisis de ubicaciones inmobiliarias 
            que proporciona insights sobre las características y ventajas de diferentes zonas. 
            Debes analizar la dinámica del mercado en cada ubicación y ofrecer perspectivas sobre 
            el potencial de inversión.""",
            
            "analisis_precio": """Eres un analista de mercado inmobiliario especializado en 
            análisis comparativo de precios y valoración de propiedades. Tu objetivo es proporcionar 
            análisis detallados sobre precios, tendencias y oportunidades de inversión.""",
            
            "detalles_propiedad": """Eres un especialista en evaluación de propiedades que analiza 
            características técnicas y funcionales de inmuebles. Debes proporcionar insights sobre 
            la calidad, distribución y potencial de las propiedades.""",
            
            "clasificacion_categoria": """Eres un consultor inmobiliario experto en categorización 
            y análisis de diferentes tipos de proyectos y propiedades. Tu objetivo es ayudar a 
            entender las diferencias y ventajas de cada categoría.""",
            
            "cronologia": """Eres un analista de tendencias del mercado inmobiliario que evalúa 
            patrones temporales y su impacto en el valor de las propiedades. Debes identificar 
            tendencias y proyecciones futuras."""
        }
        
        return prompts.get(categoria, """Eres un experto en el mercado inmobiliario que proporciona 
        análisis detallados y recomendaciones basadas en datos concretos.""")

    def generar_respuesta(self, pregunta: str) -> str:
        """Genera una respuesta basada en la pregunta del usuario."""
        try:
            # Identificar categoría
            categoria = self._identificar_categoria(pregunta)
            self.logger.info(f"Pregunta recibida: '{pregunta}' - Categoría: {categoria}")
            
            # Obtener datos según la categoría
            datos = self._obtener_datos(pregunta, categoria)
            
            # Generar respuesta con GPT
            respuesta = self._generar_respuesta_gpt(pregunta, datos, categoria)
            
            self.logger.info("Respuesta generada exitosamente")
            return respuesta
            
        except Exception as e:
            self.logger.error(f"Error al generar respuesta: {str(e)}")
            return f"Lo siento, ocurrió un error al procesar tu pregunta: {str(e)}"

    def _identificar_categoria(self, pregunta: str) -> str:
        """Identifica la categoría de la pregunta."""
        pregunta = pregunta.lower()
        mejor_match = None
        max_coincidencias = 0
        
        for categoria, palabras_clave in self.categorias.items():
            coincidencias = sum(1 for palabra in palabras_clave if palabra.lower() in pregunta)
            if coincidencias > max_coincidencias:
                max_coincidencias = coincidencias
                mejor_match = categoria
        
        self.logger.info(f"Categoría identificada: {mejor_match or 'general'} con {max_coincidencias} coincidencias")
        return mejor_match or "general"

    def _obtener_datos(self, pregunta: str, categoria: str) -> Dict[str, Any]:
        """Obtiene los datos relevantes según la categoría de la pregunta."""
        resultados = {}
        
        if categoria in self.consultas:
            for nombre_consulta, consulta in self.consultas[categoria].items():
                try:
                    # Si la consulta necesita una condición, procesarla
                    if "{condition}" in consulta:
                        consulta = self._procesar_condiciones(consulta, pregunta)
                    
                    self.logger.info(f"Ejecutando consulta {nombre_consulta}")
                    df = pd.read_sql_query(consulta, self.conn)
                    resultados[nombre_consulta] = df.to_dict('records')
                    
                except Exception as e:
                    self.logger.error(f"Error en consulta {nombre_consulta}: {str(e)}")
                    resultados[nombre_consulta] = {"error": str(e)}
        
        return resultados

    def _generar_respuesta_gpt(self, pregunta: str, datos: Dict[str, Any], categoria: str) -> str:
        """Genera una respuesta usando GPT basada en los datos obtenidos."""
        try:
            prompt = self._construir_prompt(pregunta, datos, categoria)
            
            self.logger.info("Generando respuesta con GPT")
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": self._get_system_prompt(categoria)},
                    {"role": "user", "content": prompt}
                ]
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            self.logger.error(f"Error al generar respuesta con GPT: {str(e)}")
            return self._format_raw_response(datos)

    def _procesar_condiciones(self, consulta: str, pregunta: str) -> str:
        """Procesa las condiciones de la consulta según la pregunta."""
        conditions = []
        
        # Procesar ubicación
        ubicaciones = self._extraer_ubicaciones(pregunta)
        if ubicaciones:
            ubicaciones_str = ", ".join([f"'{u}'" for u in ubicaciones])
            conditions.append(f"location IN ({ubicaciones_str})")
        
        # Procesar tipo de propiedad
        tipos = self._extraer_tipos_propiedad(pregunta)
        if tipos:
            tipos_str = ", ".join([f"'{t}'" for t in tipos])
            conditions.append(f"property_type IN ({tipos_str})")
        
        # Si no hay condiciones específicas, no filtrar
        if not conditions:
            return consulta.replace("WHERE {condition}", "")
        
        return consulta.replace("{condition}", " AND ".join(conditions))

    def cerrar(self):
        """Cierra la conexión a la base de datos."""
        try:
            self.conn.close()
            self.logger.info("Conexión a la base de datos cerrada correctamente")
        except Exception as e:
            self.logger.error(f"Error al cerrar la conexión: {str(e)}")

if __name__ == "__main__":
    # Configurar la API key de OpenAI
    openai.api_key = 'tu_api_key'
    
    # Inicializar el analizador
    analyzer = RealEstateAnalyzer(
        db_path="db.sqlite3",
        log_path="real_estate_logs"
    )

    # Lista de preguntas para análisis inmobiliario
    preguntas = [
        # Identificación y Localización
        "¿Cuál es el precio promedio de los apartamentos por zona?",
        "¿Qué características distintivas tiene la ubicación en términos de precios y tipos de propiedades?",
        
        # Análisis de Precio y Espacio
        "¿Cómo se comparan los precios por metro cuadrado entre diferentes zonas?",
        "¿Qué propiedades ofrecen la mejor relación precio-espacio en cada ubicación?",
        
        # Detalles de la Propiedad
        "¿Cuál es la distribución típica de habitaciones por tipo de propiedad?",
        "¿Qué características tienen las propiedades más valoradas en el mercado?",
        
        # Clasificación y Categoría
        "¿Cómo se clasifican los diferentes proyectos y cuál es su distribución?",
        "¿Qué tipos de propiedades tienen mejor valoración en el mercado?",
        
        # Cronología
        "¿Cómo han evolucionado los precios en los últimos meses?",
        "¿Qué tendencias se observan en el mercado inmobiliario?"
    ]

    # Procesar cada pregunta
    for pregunta in preguntas:
        print(f"\n👤 Pregunta: {pregunta}")
        respuesta = analyzer.generar_respuesta(pregunta)
        print(f"\n🏠 Respuesta:\n{respuesta}")
        print("-" * 80)

    # Cerrar el analizador
    analyzer.cerrar()

2024-12-06 14:47:10,015 [INFO] Conexión exitosa a la base de datos: db.sqlite3
2024-12-06 14:47:10,026 [INFO] Categoría identificada: identificacion_localizacion con 1 coincidencias
2024-12-06 14:47:10,029 [INFO] Pregunta recibida: '¿Cuál es el precio promedio de los apartamentos por zona?' - Categoría: identificacion_localizacion
2024-12-06 14:47:10,031 [INFO] Ejecutando consulta info_basica
2024-12-06 14:47:10,047 [INFO] Ejecutando consulta analisis_ubicacion
2024-12-06 14:47:10,057 [INFO] Generando respuesta con GPT



👤 Pregunta: ¿Cuál es el precio promedio de los apartamentos por zona?


2024-12-06 14:47:37,357 [INFO] Respuesta generada exitosamente
2024-12-06 14:47:37,359 [INFO] Categoría identificada: analisis_precio con 1 coincidencias
2024-12-06 14:47:37,360 [INFO] Pregunta recibida: '¿Qué características distintivas tiene la ubicación en términos de precios y tipos de propiedades?' - Categoría: analisis_precio
2024-12-06 14:47:37,362 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:47:37,367 [INFO] Ejecutando consulta analisis_competitividad
2024-12-06 14:47:37,371 [INFO] Ejecutando consulta analisis_tendencia
2024-12-06 14:47:37,378 [INFO] Generando respuesta con GPT



🏠 Respuesta:
De acuerdo a los datos disponibles, el precio promedio de los apartamentos varía dependiendo de la zona así:

1. En "Cerrito y Alvear, Recoleta, Capital Federal" el precio promedio de los apartamentos es de 750,000 USD.
2. En "Segui al 3500, Palermo Chico, Palermo" el precio promedio de los apartamentos es de 360,000 USD.
3. En "Belén, Belén San Bernardo, Medellín" el precio promedio de los apartamentos es de 179,254 USD.
4. En "Avenida Vieira Souto - Ipanema, Río de Janeiro, RJ" el precio promedio de los apartamentos es de 1,796,735.93 USD.

Es relevante notar que los precios varían notablemente entre las zonas. Estas diferencias podrían estar asociadas a múltiples factores como: la oferta y demanda local, el nivel socioeconómico de la zona, las características de las propiedades (como tamaño y acabados), entre otros.

Además, el precio promedio por metro cuadrado también varía, siendo "Avenida Vieira Souto - Ipanema, Río de Janeiro, RJ" la zona con mayor precio por metr

2024-12-06 14:47:56,476 [INFO] Respuesta generada exitosamente
2024-12-06 14:47:56,478 [INFO] Categoría identificada: analisis_precio con 2 coincidencias
2024-12-06 14:47:56,479 [INFO] Pregunta recibida: '¿Cómo se comparan los precios por metro cuadrado entre diferentes zonas?' - Categoría: analisis_precio
2024-12-06 14:47:56,480 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:47:56,484 [INFO] Ejecutando consulta analisis_competitividad
2024-12-06 14:47:56,489 [INFO] Ejecutando consulta analisis_tendencia
2024-12-06 14:47:56,494 [INFO] Generando respuesta con GPT


...


🏠 Respuesta:
Analizando la información disponible sobre los precios de las propiedades, el tipo de propiedad que tiene una mejor valoración en el mercado son los apartamentos. Al considerar el precio medio por metro cuadrado (precio_m2), podemos observar que los apartamentos en la Avenida Vieira Souto - Ipanema, Río de Janeiro, presentan la mayor valoración con 9406.99 por metro cuadrado, seguidos de los ubicados en la zona de Cerrito y Alvear, Recoleta, Capital Federal con un precio medio de 3000.0 por metro cuadrado. 

Es importante destacar que estos valores son significativamente superiores al promedio general de los otros lugares registrados. De los precios disponibles, el costo promedio por metro cuadrado es de 4073.71, lo que destaca aún más el valor de las propiedades en estas dos áreas.

Además, al observar el análisis de competitividad, podemos confirmar que dichos precios son competitivos en su propia zona, ya que la diferencia porcentual entre el precio por metro cuadrado 

2024-12-06 14:50:27,074 [INFO] Respuesta generada exitosamente
2024-12-06 14:50:27,096 [INFO] Categoría identificada: analisis_precio con 1 coincidencias
2024-12-06 14:50:27,112 [INFO] Pregunta recibida: '¿Qué tendencias se observan en el mercado inmobiliario?' - Categoría: analisis_precio
2024-12-06 14:50:27,169 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:50:27,245 [INFO] Ejecutando consulta analisis_competitividad



🏠 Respuesta:
De acuerdo con los datos disponibles, parece que los precios han variado bastante dependiendo de la ubicación de la propiedad.

En Avenida Vieira Souto - Ipanema, Río de Janeiro, RJ, el precio promedio de un apartamento es de 1796735.93 USD, que también es su mínimo y máximo precio. Esto indica la falta de variabilidad en el precio, tal vez debido a la falta de competencia.

En Cerrito y Alvear, Recoleta, Capital Federal, el precio promedio de un apartamento es significativamente menor, a 750000.0 USD, que una vez más es también el mínimo y máximo precio de la propiedad en esta ubicación.

Las propiedades en Segui al 3500, Palermo Chico, Palermo, son aún más asequibles, con un precio promedio de 360000.0 USD. Esto también se refleja en el precio por metro cuadrado, que es de 3529.0 USD comparativamente menor que otros lugares.

Av Bolivar 959 también tiene propiedades asequibles, con un precio promedio de 239100.0USD y un precio por metro cuadrado de 3046.64 USD. Sin emba

2024-12-06 14:50:27,303 [INFO] Ejecutando consulta analisis_tendencia
2024-12-06 14:50:27,363 [INFO] Generando respuesta con GPT
2024-12-06 14:50:54,797 [INFO] Respuesta generada exitosamente
2024-12-06 14:50:54,838 [INFO] Conexión a la base de datos cerrada correctamente



🏠 Respuesta:
Basándome en los datos proporcionados, las siguientes tendencias se observan en el mercado inmobiliario:

1. Diversidad de precios por metro cuadrado (m2) a nivel global: Los precios por metro cuadrado de los apartamentos varían significativamente dependiendo de la ubicación. Por ejemplo, el precio promedio por m2 más alto se encuentra en la Avenida Vieira Souto - Ipanema, Río de Janeiro, con un valor de 9406.99, mientras que en Belén, Belén San Bernardo, Medellín, el precio promedio por m2 es de 1884.9.

2. Competitividad de precios equilibrada: En todas las ubicaciones proporcionadas, la diferencia porcentual con respecto al precio promedio de la zona es del 0%. Esto indica una competencia equilibrada en términos de precios.

3. Tendencia de precios constante: No se observan cambios significativos en los precios promedio y los precios promedio por m2 en el ultimo periodo del año 2024, lo que sugiere una tendencia de precios estable en estas ubicaciones.

Dado esto, aquí

In [ ]:
import sqlite3
import json
import pandas as pd
import os
import logging
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import openai
from termcolor import colored


class RealEstateAnalyzer:
    def __init__(self, db_path: str, log_path: str = "logs"):
        """Inicializa el analizador de propiedades inmobiliarias."""
        self._setup_logging(log_path)
        self._validate_and_connect_db(db_path)
        self.schema = self._get_schema()
        self._init_query_mappings()

    def _validate_and_connect_db(self, db_path: str):
        """Valida y establece la conexión a la base de datos."""
        try:
            if not os.path.exists(db_path):
                self.logger.error(f"Base de datos no encontrada en: {db_path}")
                raise FileNotFoundError(f"Base de datos no encontrada en: {db_path}")
            
            self.conn = sqlite3.connect(db_path)
            self.logger.info(f"Conexión exitosa a la base de datos: {db_path}")
            
            cursor = self.conn.cursor()
            cursor.execute("""
                SELECT name FROM sqlite_master 
                WHERE type='table' AND name='chat_property';
            """)
            
            if not cursor.fetchone():
                self.logger.error("La tabla chat_property no existe en la base de datos")
                raise Exception("La tabla chat_property no existe en la base de datos")
            
            cursor.execute("PRAGMA table_info(chat_property)")
            columns = {col[1] for col in cursor.fetchall()}
            required_columns = {
                'id', 'location', 'price', 'square_meters', 'property_type',
                'num_bedrooms', 'num_rooms', 'project_type', 'image', 'url'
            }
            
            missing_columns = required_columns - columns
            if missing_columns:
                self.logger.error(f"Columnas faltantes en chat_property: {missing_columns}")
                raise Exception(f"Columnas faltantes en chat_property: {missing_columns}")
                
        except sqlite3.Error as e:
            self.logger.error(f"Error SQLite al conectar con la base de datos: {str(e)}")
            raise
        except Exception as e:
            self.logger.error(f"Error al validar la base de datos: {str(e)}")
            raise

    def _setup_logging(self, log_path: str):
        """Configura el sistema de logging."""
        if not os.path.exists(log_path):
            os.makedirs(log_path)
        
        log_file = os.path.join(log_path, f'real_estate_{datetime.now().strftime("%Y%m%d")}.log')
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s [%(levelname)s] %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def _get_schema(self) -> str:
        """Obtiene el esquema de la base de datos."""
        cursor = self.conn.cursor()
        schema_str = "Esquema de la Base de Datos:\n"
        
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' 
            AND name NOT LIKE 'sqlite_%'
            AND name NOT LIKE 'django_%'
            AND name NOT LIKE 'auth_%';
        """)
        
        for (table_name,) in cursor.fetchall():
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            
            schema_str += f"\nTabla: {table_name}\nColumnas:\n"
            for col in columns:
                schema_str += f"  - {col[1]} ({col[2]})\n"
                
        return schema_str

    def _init_query_mappings(self):
        """Inicializa las categorías y consultas predefinidas."""
        self.categorias = {
            "identificacion_localizacion": [
                "identificador", "id", "ubicacion", "localización", "ciudad",
                "provincia", "pais", "zona", "barrio", "sector"
            ],
            "analisis_precio": [
                "precio", "valor", "costo", "mercado", "comparable",
                "metro cuadrado", "competitivo", "valoración", "tasacion",
                "avaluo", "comprable", "similar"
            ],
            "detalles_propiedad": [
                "habitaciones", "dormitorios", "cuartos", "espacios",
                "distribución", "características", "condición", "baños",
                "features", "amenities", "comodidades"
            ],
            "clasificacion_categoria": [
                "categoría", "tipo", "proyecto", "residencial", "comercial",
                "lujo", "económico", "construcción", "remodelación",
                "clase", "premium", "estándar"
            ],
            "cronologia": [
                "tiempo", "mercado", "fecha", "histórico", "antigüedad",
                "created", "cuando", "desde", "hasta", "periodo"
            ]
        }

        self.consultas = {
            "identificacion_localizacion": {
                "info_basica": """
                    SELECT 
                        id,
                        location,
                        property_type,
                        project_category,
                        created_at,
                        ROUND(price, 2) as price,
                        square_meters,
                        image,
                        url,
                        description
                    FROM chat_property
                    WHERE {condition}
                """,
                "analisis_ubicacion": """
                    SELECT 
                        location,
                        COUNT(*) as total_propiedades,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(DISTINCT property_type) as tipos_propiedad,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY location
                    ORDER BY total_propiedades DESC
                """
            },
            "analisis_precio": {
                "comparativa_precios": """
                    SELECT 
                        location,
                        property_type,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(MIN(price), 2) as precio_minimo,
                        ROUND(MAX(price), 2) as precio_maximo,
                        ROUND(AVG(price/square_meters), 2) as precio_m2,
                        COUNT(*) as total_propiedades,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY location, property_type
                    ORDER BY precio_promedio DESC
                """,
                "analisis_competitividad": """
                    SELECT 
                        p1.property_type,
                        p1.location,
                        ROUND(p1.price, 2) as price,
                        p1.square_meters,
                        ROUND(p1.price/p1.square_meters, 2) as precio_m2,
                        ROUND(AVG(p2.price/p2.square_meters), 2) as precio_m2_promedio_zona,
                        ROUND((p1.price/p1.square_meters - AVG(p2.price/p2.square_meters)) / 
                            AVG(p2.price/p2.square_meters) * 100, 2) as diferencia_porcentual,
                        p1.image,
                        p1.url
                    FROM chat_property p1
                    LEFT JOIN chat_property p2 
                    ON p2.location = p1.location 
                    AND p2.property_type = p1.property_type
                    GROUP BY p1.id
                    ORDER BY p1.location, p1.property_type
                """,
                "analisis_tendencia": """
                    SELECT 
                        strftime('%Y-%m', created_at) as periodo,
                        location,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        COUNT(*) as total_propiedades,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY periodo, location
                    ORDER BY periodo DESC, precio_promedio DESC
                """
            },
            "detalles_propiedad": {
                "analisis_espacios": """
                    SELECT 
                        property_type,
                        num_bedrooms,
                        num_rooms,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY property_type, num_bedrooms, num_rooms
                    ORDER BY property_type, num_bedrooms
                """,
                "detalles_completos": """
                    SELECT 
                        id,
                        property_type,
                        location,
                        num_bedrooms,
                        num_rooms,
                        square_meters,
                        description,
                        project_category,
                        project_type,
                        residence_type,
                        ROUND(price, 2) as price,
                        ROUND(price/square_meters, 2) as precio_m2,
                        created_at,
                        image,
                        url
                    FROM chat_property
                    WHERE {condition}
                """
            },
            "clasificacion_categoria": {
                "distribucion_categorias": """
                    SELECT 
                        project_category,
                        project_type,
                        residence_type,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY project_category, project_type, residence_type
                    HAVING cantidad > 0
                    ORDER BY cantidad DESC
                """,
                "analisis_tipos": """
                    SELECT 
                        property_type,
                        project_type,
                        COUNT(*) as cantidad,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        ROUND(MIN(price), 2) as precio_minimo,
                        ROUND(MAX(price), 2) as precio_maximo,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY property_type, project_type
                    HAVING cantidad > 0
                    ORDER BY cantidad DESC
                """
            },
            "cronologia": {
                "analisis_temporal": """
                    SELECT 
                        strftime('%Y-%m', created_at) as mes,
                        COUNT(*) as nuevas_propiedades,
                        ROUND(AVG(price), 2) as precio_promedio,
                        ROUND(AVG(square_meters), 2) as metros_promedio,
                        COUNT(DISTINCT property_type) as tipos_diferentes,
                        ROUND(AVG(price/square_meters), 2) as precio_m2_promedio,
                        GROUP_CONCAT(DISTINCT CASE WHEN image != '' THEN image ELSE NULL END) as imagenes,
                        GROUP_CONCAT(DISTINCT CASE WHEN url != '' THEN url ELSE NULL END) as urls
                    FROM chat_property
                    GROUP BY mes
                    ORDER BY mes DESC
                """
            }
        }

    def _extraer_ubicaciones(self, pregunta: str) -> List[str]:
        """Extrae las ubicaciones mencionadas en la pregunta."""
        ubicaciones_conocidas = [
            "Mar de Plata", "Buenos Aires", "Villavicencio", "Medellin",
            "Recoleta", "Palermo", "Belgrano", "Mar del Plata"
        ]
        
        found = []
        pregunta_lower = pregunta.lower()
        for ubicacion in ubicaciones_conocidas:
            if ubicacion.lower() in pregunta_lower:
                found.append(ubicacion)
        
        return found

    def _extraer_tipos_propiedad(self, pregunta: str) -> List[str]:
        """Extrae los tipos de propiedad mencionados en la pregunta."""
        tipos_conocidos = {
            "apartamento": "apartment",
            "casa": "house",
            "cabaña": "cabin",
            "departamento": "apartment",
            "oficina": "office",
            "local": "commercial",
            "propiedad": "property"
        }
        
        found = []
        pregunta_lower = pregunta.lower()
        for tipo_es, tipo_en in tipos_conocidos.items():
            if tipo_es in pregunta_lower:
                found.append(tipo_en)
        
        return found

    def _format_raw_response(self, datos: Dict[str, Any]) -> str:
        """Formatea los datos en caso de error con GPT."""
        response = "📊 Datos encontrados:\n\n"
        
        for nombre_consulta, resultados in datos.items():
            if isinstance(resultados, list) and resultados:
                response += f"📍 {nombre_consulta.replace('_', ' ').title()}:\n"
                for item in resultados[:5]:
                    # Formatear datos principales
                    data_str = {k: v for k, v in item.items() if k not in ['image', 'url', 'imagenes', 'urls']}
                    response += f"  • {str(data_str)}\n"
                    
                    # Agregar información de imágenes
                    if 'image' in item and item['image']:
                        response += f"    📸 Imagen: {item['image']}\n"
                    elif 'imagenes' in item and item['imagenes']:
                        response += f"    📸 Imágenes: {item['imagenes']}\n"
                        
                    # Agregar información de URLs
                    if 'url' in item and item['url']:
                        response += f"    🔗 URL: {item['url']}\n"
                    elif 'urls' in item and item['urls']:
                        response += f"    🔗 URLs: {item['urls']}\n"
                    
                    response += "\n"
            elif isinstance(resultados, dict) and "error" in resultados:
                response += f"❌ Error en {nombre_consulta}: {resultados['error']}\n\n"
        
        return response.strip()

    def _construir_prompt(self, pregunta: str, datos: Dict[str, Any], categoria: str) -> str:
        """Construye el prompt para GPT basado en los datos."""
        prompt = f"""
        Pregunta del usuario: {pregunta}
        
        Categoría identificada: {categoria}
        
        Datos disponibles:
        """
        
        for nombre_consulta, resultados in datos.items():
            prompt += f"\n{nombre_consulta.upper()}:\n"
            if isinstance(resultados, list):
                for item in resultados[:5]:
                    # Crear una copia del item sin los campos de imagen para evitar strings muy largos
                    item_info = {k: v for k, v in item.items() if k not in ['image', 'imagenes']}
                    prompt += json.dumps(item_info, ensure_ascii=False, indent=2) + "\n"
                    
                    # Incluir referencias a recursos multimedia
                    if any(k in item and item[k] for k in ['url', 'urls']):
                        prompt += "Referencias disponibles:\n"
                        if item.get('url'):
                            prompt += f"- URL de la propiedad: {item['url']}\n"
                        elif item.get('urls'):
                            prompt += f"- URLs relacionadas: {item['urls']}\n"
                        
            elif isinstance(resultados, dict) and "error" in resultados:
                prompt += f"Error: {resultados['error']}\n"
        
        prompt += """
        Por favor, proporciona un análisis detallado que:
        1. Responda directamente a la pregunta del usuario
        2. Destaque insights relevantes de los datos
        3. Proporcione contexto cuando sea necesario
        4. Ofrezca recomendaciones basadas en los datos
        5. Incluya métricas específicas cuando estén disponibles
        6. Compare con promedios o referencias del mercado cuando sea posible
        7. Mencione la disponibilidad de imágenes y recursos adicionales cuando existan
        """
        
        return prompt

    def generar_respuesta(self, pregunta: str) -> str:
        """Genera una respuesta basada en la pregunta del usuario."""
        try:
            # Identificar categoría
            categoria = self._identificar_categoria(pregunta)
            self.logger.info(f"Pregunta recibida: '{pregunta}' - Categoría: {categoria}")
            
            # Obtener datos según la categoría
            datos = self._obtener_datos(pregunta, categoria)
            
            # Generar respuesta con GPT
            respuesta = self._generar_respuesta_gpt(pregunta, datos, categoria)
            
            self.logger.info("Respuesta generada exitosamente")
            return respuesta
            
        except Exception as e:
            self.logger.error(f"Error al generar respuesta: {str(e)}")
            return f"Lo siento, ocurrió un error al procesar tu pregunta: {str(e)}"

    def _identificar_categoria(self, pregunta: str) -> str:
        """Identifica la categoría de la pregunta."""
        pregunta = pregunta.lower()
        mejor_match = None
        max_coincidencias = 0
        
        for categoria, palabras_clave in self.categorias.items():
            coincidencias = sum(1 for palabra in palabras_clave if palabra.lower() in pregunta)
            if coincidencias > max_coincidencias:
                max_coincidencias = coincidencias
                mejor_match = categoria
        
        self.logger.info(f"Categoría identificada: {mejor_match or 'general'} con {max_coincidencias} coincidencias")
        return mejor_match or "general"

    def _obtener_datos(self, pregunta: str, categoria: str) -> Dict[str, Any]:
        """Obtiene los datos relevantes según la categoría de la pregunta."""
        resultados = {}
        
        if categoria in self.consultas:
            for nombre_consulta, consulta in self.consultas[categoria].items():
                try:
                    # Si la consulta necesita una condición, procesarla
                    if "{condition}" in consulta:
                        consulta = self._procesar_condiciones(consulta, pregunta)
                    
                    self.logger.info(f"Ejecutando consulta {nombre_consulta}")
                    df = pd.read_sql_query(consulta, self.conn)
                    resultados[nombre_consulta] = df.to_dict('records')
                    
                except Exception as e:
                    self.logger.error(f"Error en consulta {nombre_consulta}: {str(e)}")
                    resultados[nombre_consulta] = {"error": str(e)}
        
        return resultados

    def _generar_respuesta_gpt(self, pregunta: str, datos: Dict[str, Any], categoria: str) -> str:
        """Genera una respuesta usando GPT basada en los datos obtenidos."""
        try:
            prompt = self._construir_prompt(pregunta, datos, categoria)
            
            self.logger.info("Generando respuesta con GPT")
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": self._get_system_prompt(categoria)},
                    {"role": "user", "content": prompt}
                ]
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            self.logger.error(f"Error al generar respuesta con GPT: {str(e)}")
            return self._format_raw_response(datos)

    def _procesar_condiciones(self, consulta: str, pregunta: str) -> str:
        """Procesa las condiciones de la consulta según la pregunta."""
        conditions = []
        
        # Procesar ubicación
        ubicaciones = self._extraer_ubicaciones(pregunta)
        if ubicaciones:
            ubicaciones_str = ", ".join([f"'{u}'" for u in ubicaciones])
            conditions.append(f"location IN ({ubicaciones_str})")
        
        # Procesar tipo de propiedad
        tipos = self._extraer_tipos_propiedad(pregunta)
        if tipos:
            tipos_str = ", ".join([f"'{t}'" for t in tipos])
            conditions.append(f"property_type IN ({tipos_str})")
        
        # Si no hay condiciones específicas, no filtrar
        if not conditions:
            return consulta.replace("WHERE {condition}", "")
        
        return consulta.replace("{condition}", " AND ".join(conditions))

    def _get_system_prompt(self, categoria: str) -> str:
        """Obtiene el prompt de sistema según la categoría."""
        prompts = {
            "identificacion_localizacion": """Eres un experto en análisis de ubicaciones inmobiliarias 
            que proporciona insights sobre las características y ventajas de diferentes zonas. 
            Debes analizar la dinámica del mercado en cada ubicación y ofrecer perspectivas sobre 
            el potencial de inversión. Cuando haya imágenes o recursos adicionales disponibles, 
            menciónalos para enriquecer tu análisis.""",
            
            "analisis_precio": """Eres un analista de mercado inmobiliario especializado en 
            análisis comparativo de precios y valoración de propiedades. Tu objetivo es proporcionar 
            análisis detallados sobre precios, tendencias y oportunidades de inversión. Incluye 
            referencias a recursos visuales cuando estén disponibles.""",
            
            "detalles_propiedad": """Eres un especialista en evaluación de propiedades que analiza 
            características técnicas y funcionales de inmuebles. Debes proporcionar insights sobre 
            la calidad, distribución y potencial de las propiedades. Aprovecha las imágenes y 
            recursos disponibles para respaldar tus análisis.""",
            
            "clasificacion_categoria": """Eres un consultor inmobiliario experto en categorización 
            y análisis de diferentes tipos de proyectos y propiedades. Tu objetivo es ayudar a 
            entender las diferencias y ventajas de cada categoría. Utiliza los recursos visuales 
            disponibles para ilustrar tus puntos.""",
            
            "cronologia": """Eres un analista de tendencias del mercado inmobiliario que evalúa 
            patrones temporales y su impacto en el valor de las propiedades. Debes identificar 
            tendencias y proyecciones futuras. Incorpora referencias a recursos multimedia cuando 
            sea relevante."""
        }
        
        return prompts.get(categoria, """Eres un experto en el mercado inmobiliario que proporciona 
        análisis detallados y recomendaciones basadas en datos concretos. Aprovecha todos los 
        recursos disponibles, incluyendo imágenes y referencias externas, para enriquecer tus análisis.""")

    def cerrar(self):
        """Cierra la conexión a la base de datos."""
        try:
            self.conn.close()
            self.logger.info("Conexión a la base de datos cerrada correctamente")
        except Exception as e:
            self.logger.error(f"Error al cerrar la conexión: {str(e)}")

if __name__ == "__main__":
    # Configurar la API key de OpenAI
    openai.api_key = 'tu_api_key'
    
    # Inicializar el analizador
    analyzer = RealEstateAnalyzer(
        db_path="db.sqlite3",
        log_path="real_estate_logs"
    )

    # Lista de preguntas para análisis inmobiliario
    preguntas = [
        # Identificación y Localización
        "¿Cuál es el precio promedio de los apartamentos por zona?",
        "¿Qué características distintivas tiene la ubicación en términos de precios y tipos de propiedades?",
        
        # Análisis de Precio y Espacio
        "¿Cómo se comparan los precios por metro cuadrado entre diferentes zonas?",
        "¿Qué propiedades ofrecen la mejor relación precio-espacio en cada ubicación?",
        
        # Detalles de la Propiedad
        "¿Cuál es la distribución típica de habitaciones por tipo de propiedad?",
        "¿Qué características tienen las propiedades más valoradas en el mercado?",
        
        # Clasificación y Categoría
        "¿Cómo se clasifican los diferentes proyectos y cuál es su distribución?",
        "¿Qué tipos de propiedades tienen mejor valoración en el mercado?",
        
        # Cronología
        "¿Cómo han evolucionado los precios en los últimos meses?",
        "¿Qué tendencias se observan en el mercado inmobiliario?"
    ]

    # Procesar cada pregunta
    for pregunta in preguntas:
        print(f"\n👤 Pregunta: {pregunta}")
        respuesta = analyzer.generar_respuesta(pregunta)
        print(f"\n🏠 Respuesta:\n{respuesta}")
        print("-" * 80)

    # Cerrar el analizador
    analyzer.cerrar()

2024-12-06 14:52:15,462 [INFO] Conexión exitosa a la base de datos: db.sqlite3
2024-12-06 14:52:15,480 [INFO] Categoría identificada: identificacion_localizacion con 1 coincidencias
2024-12-06 14:52:15,484 [INFO] Pregunta recibida: '¿Cuál es el precio promedio de los apartamentos por zona?' - Categoría: identificacion_localizacion
2024-12-06 14:52:15,489 [INFO] Ejecutando consulta info_basica
2024-12-06 14:52:15,548 [INFO] Ejecutando consulta analisis_ubicacion
2024-12-06 14:52:15,567 [INFO] Generando respuesta con GPT



👤 Pregunta: ¿Cuál es el precio promedio de los apartamentos por zona?


2024-12-06 14:52:44,910 [INFO] Respuesta generada exitosamente
2024-12-06 14:52:44,917 [INFO] Categoría identificada: analisis_precio con 1 coincidencias
2024-12-06 14:52:44,924 [INFO] Pregunta recibida: '¿Qué características distintivas tiene la ubicación en términos de precios y tipos de propiedades?' - Categoría: analisis_precio
2024-12-06 14:52:44,927 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:52:44,948 [INFO] Ejecutando consulta analisis_competitividad
2024-12-06 14:52:44,989 [INFO] Ejecutando consulta analisis_tendencia



🏠 Respuesta:
Basado en los datos disponibles, a continuación se presentan los precios promedio de los apartamentos por zona:

1. Cerrito y Alvear, Recoleta, Capital Federal: El precio promedio en esta zona es de $750,000 con una proporción precio por metro cuadrado de $3,000/m2. Esto se basa en un solo edificio de lujo en la base de datos que mide 250 m2.

2. Segui al 3500, Palermo Chico, Palermo: El precio promedio en esta ubicación es de $360,000 con una proporción de $3,529/m2. La propiedad es un apartamento de lujo de 102 m2. Esta propiedad puede ser vista en (`https://www.zonaprop.com.ar/propiedades/clasificado/veclapin-piso-alto-reciclado-cochera!-54456943.html`)[https://www.zonaprop.com.ar/propiedades/clasificado/veclapin-piso-alto-reciclado-cochera!-54456943.html].

3. Avenida Vieira Souto - Ipanema, Río de Janeiro: Aquí, el precio promedio es de $1,796,735.93 (USD), con un precio por metro cuadrado de $9,406.99/m2. Esta es una propiedad premium con 191 m2. Puede obtener más i

2024-12-06 14:52:45,192 [INFO] Generando respuesta con GPT
2024-12-06 14:53:10,355 [INFO] Respuesta generada exitosamente
2024-12-06 14:53:10,358 [INFO] Categoría identificada: analisis_precio con 2 coincidencias
2024-12-06 14:53:10,360 [INFO] Pregunta recibida: '¿Cómo se comparan los precios por metro cuadrado entre diferentes zonas?' - Categoría: analisis_precio
2024-12-06 14:53:10,362 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:53:10,368 [INFO] Ejecutando consulta analisis_competitividad
2024-12-06 14:53:10,379 [INFO] Ejecutando consulta analisis_tendencia
2024-12-06 14:53:10,392 [INFO] Generando respuesta con GPT


...


🏠 Respuesta:
Según los datos disponibles, la mejor valoración de propiedades se observa en los apartamentos. Los precios y la valoración varían según la ubicación y las características del edificio o la propiedad.

1. El apartamento de mayor valoración se encuentra en la Avenida Vieira Souto - Ipanema, en Río de Janeiro, con un promedio de precio de 1796735.93, y un precio por metro cuadrado de 9406.99. Más detalles de esta propiedad se pueden encontrar en [este enlace](https://www.apartamentosrio.com.br/imovel/venda+apartamento+3-quartos+ipanema+rio-de-janeiro+rj+191m2+rs10800000/217).

2. La propiedad en Cerrito y Alvear, en Recoleta, Capital Federal, tiene un promedio de precio de 750000.0, con un precio por metro cuadrado de 3000.0. Sin embargo, no se dispone de un enlace para obtener más detalles de esta propiedad.

3. La propiedad en Segui al 3500, en Palermo Chico, Palermo, presenta un promedio de precio de 360000.0 y un precio por metro cuadrado de 3529.0. Más detalles de esta

2024-12-06 14:56:18,217 [INFO] Respuesta generada exitosamente
2024-12-06 14:56:18,222 [INFO] Categoría identificada: analisis_precio con 1 coincidencias
2024-12-06 14:56:18,226 [INFO] Pregunta recibida: '¿Qué tendencias se observan en el mercado inmobiliario?' - Categoría: analisis_precio
2024-12-06 14:56:18,230 [INFO] Ejecutando consulta comparativa_precios
2024-12-06 14:56:18,245 [INFO] Ejecutando consulta analisis_competitividad
2024-12-06 14:56:18,259 [INFO] Ejecutando consulta analisis_tendencia
2024-12-06 14:56:18,276 [INFO] Generando respuesta con GPT



🏠 Respuesta:
La evolución de los precios de las propiedades en los últimos meses ha sido específica para cada ubicación, según nuestros análisis. 

Por ejemplo, en Av Bolivar 959, el precio promedio de los apartamentos es de $239.100 con un precio por metro cuadrado de $3046.64. Este precio es consistente con el promedio de la zona, ya que no hay diferencia porcentual según nuestro análisis de competitividad. Intenta referirte a este [enlace](https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2940-amaranta-pueblo-libre-lima-lima-abril-grupo-inmobiliario#superdestaque_search) para una visión visual.

En comparación, los apartamentos en la Avenida Vieira Souto - Ipanema en Río de Janeiro tienen un precio promedio muchísimo más alto de aproximadamente $1.796.735,93 con un precio por metro cuadrado de $9406.99. Este es el precio más alto que hemos observado en todas las ubicaciones analizadas, y también es consistente con el promedio de la zona. Puedes referirte a este [enlace](ht

2024-12-06 14:56:51,753 [INFO] Respuesta generada exitosamente
2024-12-06 14:56:51,771 [INFO] Conexión a la base de datos cerrada correctamente



🏠 Respuesta:
Según los datos disponibles, se observan varias tendencias interesantes en el mercado inmobiliario. Es importante destacar que estas tendencias tienen en cuenta varias localidades y tipos diferentes de propiedades, específicamente apartamentos. 

En Río de Janeiro, Brasil, de acuerdo con el apartamento encontrado en la Avenida Vieira Souto en Ipanema, se nota un precio promedio y máximo de 1796735.93 y con un costo promedio por metro cuadrado de 9406.99; lo que indica que el mercado inmobiliario es de alto valor y costoso en esta región. Para más detalles, referirse a la [URL](https://www.apartamentosrio.com.br/imovel/venda+apartamento+3-quartos+ipanema+rio-de-janeiro+rj+191m2+rs10800000/217).

Por otro lado, en Recoleta, Capital Federal en Buenos Aires, Argentina, el precio promedio y máximo de un apartamento es considerablemente menor, estimado en 750,000 con un precio por m2 de 3000.0. Estos datos sugieren que el mercado puede ser más accesible.

En el distrito de Pale